# Intro

Server-Sent Events (SSE) are a web technology that allow servers to push real-time updates to web clients over a single HTTP connection. Unlike traditional HTTP requests where the client must repeatedly ask for new data, SSE creates a one-way channel from the server to the client that stays open, allowing the server to send updates whenever they're available.

![SSE Diagram](static_blog_imgs/sse_diag.png)

Some key characteristics of SSE:

- One-directional: Data flows only from server to client
- Uses standard HTTP (unlike WebSockets which use a different protocol)
    - WebSockets are bidirectional, meaning data can flow both ways - from server to client and from client to server over the same connection. This makes WebSockets suitable for applications requiring real-time communication in both directions, like chat applications or multiplayer games.
    - In contrast, SSE are strictly one-directional (server to client only).
    - This fundamental difference in communication flow is one of the key factors to consider when choosing between these technologies for your application.
- Automatically reconnects if the connection is lost
- Lightweight compared to alternatives like WebSockets
- Built into browsers with the EventSource API

# Example 1: Basic SSE Implementation

Let's look at a basic example first to see how SSE works.
In this example an SSE connection is setup when the page is loaded.

In [1]:
# | echo: false
from IPython.display import Markdown, display


def import_python_as_markdown(file_path):
    with open(file_path, "r") as file:
        content = file.read()
    return f"```python\n{content}\n```"


file_path = "sse1.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
# ruff: noqa: F403, F405
from asyncio import sleep

from fasthtml.common import *
from monsterui.all import *

app, rt = fast_app(
    hdrs=(
        Theme.blue.headers(highlightjs=True),  # monsterui styling
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.1/sse.js"),  # Include this to use the SSE extension
    ),
    live=True,
)


@rt("/")
def index():
    return Container(
        H3("Intro SSE Example"),
        Div(
            P("The contents of this <div> will be updated in real time with each SSE message received."),
            hx_ext="sse",  # To connect to an SSE server, use the hx_ext="sse" attribute to install the extension on that HTML element
            sse_swap="EventName",  # This default event name is "message" if we don't specify it otherwise
            sse_connect="/sse-stream",  # This is the URL of the SSE endpoint we create and connect to
            hx_swap="beforeend show:bottom",  # Determines how the content will be inserted into that target element. Here, each new message is added at the end of the div and the page automatically scrolls to show the new message
            hx_target=None,  # None is the default. By not specifying a target for the swap, it defaults to the element that triggered the request i.e. id="sse-content"
            id="sse-content",
        ),
    )


async def message_generator():
    # This sse_message function converts an HTML element into the specific format required for Server-Sent Events (SSE) streaming.
    # The first argument is an FT component (FastHTML element) that you want to send via SSE.
    # The second argument is the name of the SSE event (defaults to "message" if not specified).
    # It must match the sse_swap attribute above i.e. event="EventName"

    for i in range(10):
        yield sse_message(Div(P(f"message number {i}")), event="EventName")
        await sleep(0.5)

    yield sse_message(Div(P("DONE")), event="EventName")


@rt("/sse-stream")
async def sse_stream():
    return EventStream(message_generator())


serve(port=5010)

```

** TODO: Run The Example as A Video Or Embedded App In the BLOG!**

## Breaking Down The Example in Details

### Initial Page Load

When a user navigates to the root URL, the browser sends a `GET` request to the server.
The server responds with the HTML generated by the `index()` function, which creates:

- A container with a heading "Intro SSE Example"

- A div with ID "sse-content" containing an introductory paragraph. This div has several HTMX attributes that configure SSE behavior


- The browser renders this initial HTML, showing the heading and the introductory paragraph.

### SSE Connection Establishment

The browser sees the `hx_ext="sse"` and `sse_connect="/sse-stream"` attributes on the div and recognizes that it needs to establish a Server-Sent Events connection.
The browser automatically opens an `EventSource` connection to the `/sse-stream` endpoint.
On the server, when this connection request arrives, it triggers the `sse_stream()` function, which:

- Creates a new instance of the `message_generator()` coroutine
- Wraps it in an `EventStream` response object
- Sends the appropriate HTTP headers to establish an SSE connection


### Message Streaming Process

Once the connection is established, the `message_generator()` coroutine begins execution:
For each iteration (0-9):

- It creates an HTML message containing "message number {i}"
- Converts this to SSE format with the event name "EventName"
- Yields this message, which is immediately sent to the browser
- Pauses for 0.5 seconds using `await sleep(0.5)`
- During this pause, the server can handle other requests because of the use of `async`


After the 10 numbered messages, it sends a final message containing "DONE".

### Client-Side Processing

As each SSE message arrives at the browser, HTMX intercepts it because of the `hx_ext="sse"` attribute.
It checks the event name in the message ("EventName") and matches it against the `sse_swap="EventName"` attribute.
Since they match, HTMX processes this message


The content of each message is inserted into the div according to the `hx_swap="beforeend show:bottom"` attribute:

- `beforeend`: Each new message is added at the end of the existing content
- `show:bottom`: The page automatically scrolls to show the new content

The user sees each message appear approximately every half second, with the page scrolling to keep the latest message visible.

### Connection Behavior After Completion

After the final "DONE" message, the generator is exhausted, but the SSE connection doesn't automatically close.
The browser's `EventSource` implementation will detect the end of the stream and automatically attempt to reconnect after a brief delay.
This reconnection will trigger another call to `sse_stream()`, creating a new instance of `message_generator()`, and the sequence will start over.
This cycle will continue indefinitely, with the div accumulating more and more messages, unless the page is navigated away from or the connection is explicitly closed.

This was a surprise to me when first learning about SSE. I expected the connection to close after the initial `for` loop completed.

**TODO: Add Video of Example 2 Running. Show what button does but show that it doesn't work as expected.**

# Example 2: How NOT to Start SSE with Button Click

Next I wanted to show how to start an SSE connection with a button click, but what I thought would work didn't.
So this example shows how **NOT** to do it. It's still a learning opportunity worth documenting.

It's the same code as in Example 1 with the following changes:

- Add a form around a button and put the SSE connection attributes on the button.



In [2]:
# | echo: false
file_path = "sse2.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
# ruff: noqa: F403, F405
from asyncio import sleep

from fasthtml.common import *
from monsterui.all import *

app, rt = fast_app(
    hdrs=(
        Theme.blue.headers(highlightjs=True),
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.1/sse.js"),
    ),
    live=True,
)


@rt("/")
def index():
    return Container(
        H3("How NOT to Start SSE with Button Click"),
        Form(
            Button(
                "Start SSE",
                hx_ext="sse",
                sse_swap="EventName",
                sse_connect="/sse-stream",
                hx_swap="beforeend show:bottom",
                hx_target="#sse-content",  # This is the target element that will receive the SSE messages
            ),
        ),
        Div(
            P("The contents of this <div> will be updated in real time with each SSE message received."),
            id="sse-content",
        ),
    )


async def message_generator():
    for i in range(10):
        yield sse_message(Div(P(f"message number {i}")), event="EventName")
        await sleep(0.5)

    yield sse_message(Div(P("DONE")), event="EventName")


@rt("/sse-stream")
async def sse_stream():
    return EventStream(message_generator())


serve(port=5010)

```



In this example, the SSE stream starts before the button is clicked because of how the HTMX SSE extension works with the attributes on the button.
The key issue is that the SSE connection attributes are **directly on the button element**:

```python
Button(
    "Start SSE",
    hx_ext="sse",
    sse_swap="EventName",
    sse_connect="/sse-stream",
    hx_swap="beforeend show:bottom",
    hx_target="#sse-content",  # This is the target element that will receive the SSE messages
)
```

With this configuration, the SSE connection is established immediately when the page is loaded because:

- The HTMX SSE extension is loaded (`hx_ext="sse"`).
- The `sse_connect` attribute is present.
- When both of these conditions are met, HTMX automatically initiates the SSE connection.

This was not quite what I was going for. Clicking the button does start a new SSE connection, however
my plan was to start the SSE connection **only** when the button is clicked, not on the initial page load.
In the next example I show one way of doing this.



# Example 3: How to Start SSE with Button Click (Not On Page Load)

The main difference in this example is that 
SSE connection attributes are **not** on the main page which loads first.
Instead, when we click the button, we trigger a GET request to a new endpoint, `/start-sse`.
This endpoint returns a div with the SSE connection attributes, hence creating the initial SSE connection. 
It will replace the existing div with same id `sse-content`. The HTMX SSE connection is established and messages are streamed to the client.



**TODO: Add Video of Example 3 Running. Show what button does**

In [3]:
# | echo: false
file_path = "sse3.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
# ruff: noqa: F403, F405
from asyncio import sleep

from fasthtml.common import *
from monsterui.all import *

app, rt = fast_app(
    hdrs=(
        Theme.blue.headers(highlightjs=True),
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.1/sse.js"),
    ),
    live=True,
)


@rt("/")
def index():
    return Container(
        H3("How to Start SSE with Button Click"),
        Form(
            Button(
                "Start SSE",
                hx_get="/start-sse",
                hx_target="#sse-content",
            ),
        ),
        Div(
            P("The contents of this <div> will be updated in real time with each SSE message received."),
            id="sse-content",
        ),
    )


async def message_generator():
    for i in range(10):
        yield sse_message(Div(P(f"message number {i}")), event="EventName")
        await sleep(0.5)

    yield sse_message(Div(P("DONE")), event="EventName")


@rt("/start-sse")
def start_sse():
    return (
        Div(
            P("The contents of this <div> will be updated in real time with each SSE message received."),
            hx_ext="sse",
            sse_swap="EventName",
            sse_connect="/sse-stream",
            hx_swap="beforeend show:bottom",
            hx_target="#sse-content",
            id="sse-content",
        ),
    )


@rt("/sse-stream")
async def sse_stream():
    return EventStream(message_generator())


serve(port=5010)

```

Note that after the SSE connection is established, it stays open and the messages are streamed to the client indefinitely as in the previous examples.

# Example 4: Closing The SSE Connection Gracefully

In some cases you may want to close the SSE connection gracefully when a specific message is received.
You can use the `sse_close` attribute in this case.
This next example is the same as Example 3 except for that we add a special message to `close` the connection.
The only difference from the previous example is the two lines of code which are both commented on below.

In [4]:
# | echo: false
file_path = "sse4.py"
markdown_content = import_python_as_markdown(file_path)
display(Markdown(markdown_content))

```python
# ruff: noqa: F403, F405
from asyncio import sleep

from fasthtml.common import *
from monsterui.all import *

app, rt = fast_app(
    hdrs=(
        Theme.blue.headers(highlightjs=True),
        Script(src="https://unpkg.com/htmx-ext-sse@2.2.1/sse.js"),
    ),
    live=True,
)


@rt("/")
def index():
    return Container(
        H3("How to Start SSE with Button Click"),
        Form(
            Button(
                "Start SSE",
                hx_get="/start-sse",
                hx_target="#sse-content",
            ),
        ),
        Div(
            P("The contents of this <div> will be updated in real time with each SSE message received."),
            id="sse-content",
        ),
    )


async def message_generator():
    for i in range(10):
        yield sse_message(Div(P(f"message number {i}")), event="EventName")
        await sleep(0.5)

    yield sse_message(Div(P("DONE")), event="EventName")
    yield sse_message(Div(), event="close") # A special event message to close the connection


@rt("/start-sse")
def start_sse():
    return (
        Div(
            P("The contents of this <div> will be updated in real time with each SSE message received."),
            hx_ext="sse",
            sse_swap="EventName",
            sse_connect="/sse-stream",
            sse_close="close", # When this event is received, the SSE connection will be closed
            hx_swap="beforeend show:bottom",
            hx_target="#sse-content",
            id="sse-content",
        ),
    )


@rt("/sse-stream")
async def sse_stream():
    return EventStream(message_generator())


serve(port=5010)

```

**TODO: Add Video of Example 4 Running.**

# Example 5: Receiving Multiple Named Events

You can also listen to multiple events from a single EventSource. 
The listeners can be either:

- the same element that contains the `hx_ext` and `sse_connect` attributes
- or child elements of the element containing the `hx_ext` and `sse_connect` attributes

## Example 5a: Multiple Listeners on the Same Element

## Example 5b: Multiple Listeners on Child Elements


# Example 6: `hx_trigger` example


# Example 7: Similar to FastHTML Example with Shutdown thing

# Recap of Main Attributes for Server Sent Events in FastHTML

The [htmx.org website](https://htmx.org/extensions/sse/) has a great explanation of how to use the htmx Server Sent Event (SSE) extension. Here's a quick summary. We will use the `FastHTML` naming convention for the attributes. For example,
`hx-ext` becomes `hx_ext`, `sse-connect` becomes `sse_connect`, etc.

- `hx_ext="sse"` - This is required to use the SSE extension.
- `sse_connect="<url>"` - The URL of the SSE endpoint
- `sse_swap="<message-name>"` - The name of the message to swap into the DOM.
- `hx_trigger="sse:<message-name>"` - SSE messages can also trigger HTTP callbacks using the hx-trigger attribute.
- `sse_close=<message-name>` - To close the EventStream gracefully when that message is received. This might be helpful if you want to send information to a client that will eventually stop.


# Chat Bot Example